In [8]:
import telegram
from telegram.ext import Updater
from telegram.ext import CommandHandler
import logging

%run token.py

updater = Updater(token=token_id, use_context=True)
dispatcher = updater.dispatcher

In [9]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',level=logging.INFO)

In [10]:
start_text = '''
    Olá! \n
    Sou um bot de notícias e informações sobre a Covid-19!\n
    O serviço de jornalismo do Instituto Oswaldo Cruz fez a triagem do conteúdo aqui.
    Sou um bot repassador não oficial! Recebo deles e repasso para vocês.\n
    Não conhece o Instituto Oswaldo Cruz? \n
    Entre em www.ioc.fiocruz.br para conhecer! \n
    (Bot não-oficial administrado por Tiago Lubiana)\n

    (em desenvolvimento)\n"
'''

In [11]:
def start(update, context):
        context.bot.send_message(chat_id=update.effective_chat.id,
                                 text=start_text)
        
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [12]:
with open('14_05_2020_p1.txt', 'r') as file:
        comunicado = file.read()
        comunicado_parseado = comunicado.split("--")

In [16]:
def get_header(section):
    raw_header = section.split("\n\n")[0]
    header = raw_header.replace("\n", "")
    header = header.replace("\U0001f97c ", "")
    header = header.replace("\U0001f9ea", "")
    return(header),

def get_section_content(section):
    raw_content = section[1:]
    return(raw_content)

In [17]:
comunicado_headers = [get_header(section) for section in comunicado_parseado]
comunicado_sections = [get_section_content(section) for section in comunicado_parseado]
comunicado_header_to_section = dict(zip(comunicado_headers, comunicado_sections ))

In [18]:
comunicado_headers

[('Destaques - 14/05/2020',),
 ('📊 Epidemiologia',),
 ('🔬 Pesquisa científica',),
 (' Diagnóstico',),
 ('🏡 Distanciamento social',),
 ('🗂️ Política',),
 ('Saúde Pública',),
 ('🌐 Internacional',),
 ('🔍 Recortes da pandemia',),
 ('✅ Curtas',),
 ('Produzido pelo Serviço de Jornalismo e Comunicação do Instituto Oswaldo Cruz (IOC/Fiocruz)',)]